In [ ]:
import os
os.chdir("..")  # move up from notebooks to repo root
print("Now in:", os.getcwd())
# Add project root to sys.path so 'src' can be imported
import sys
project_root = os.path.abspath(os.getcwd())
if project_root not in sys.path:
    sys.path.append(project_root)

Now in: c:\Users\faizan\Desktop\EAST


In [ ]:
import os
print(os.getcwd())
from src.dataset import EASTDataset

ds = EASTDataset("data/icdar2015/train_images", "data/icdar2015/train_maps", size=512)

print("Dataset length:", len(ds))
print("First 3 image paths:", ds.img_paths[:3])

c:\Users\faizan\Desktop\EAST
Dataset length: 1000
First 3 image paths: ['data/icdar2015/train_images\\img_1.jpg', 'data/icdar2015/train_images\\img_10.jpg', 'data/icdar2015/train_images\\img_100.jpg']


In [ ]:
import torch
from torch.utils.data import DataLoader
from src.dataset import EASTDataset

# Create a small dataset instance
train_dataset = EASTDataset(
    img_dir="data/icdar2015/train_images",
    map_dir="data/icdar2015/train_maps",
    size=512,              # keep same as paper
    training=True          # enable random cropping/augmentation
)

# Try a small dataloader
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)

# Fetch one batch, now including the nmap
imgs, scores, geos, nmaps = next(iter(train_loader))

# Print shapes to verify
print("Images :", imgs.shape)    # expect [B, 3, 512, 512]
print("Scores :", scores.shape)  # expect [B, 1, 512, 512]
print("Geos   :", geos.shape)    # expect [B, 8, 512, 512]
print("Nmaps  :", nmaps.shape)   # expect [B, 1, 512, 512]

# Check value ranges and properties
print("Image range:", imgs.min().item(), "to", imgs.max().item())
print("Score unique values:", torch.unique(scores))
print("Geo stats: mean", geos.mean().item(), "std", geos.std().item())

# Verify nmap has non-zero values where score is positive
print("Nmap stats (on positive pixels): mean", nmaps[scores > 0].mean().item(), "std", nmaps[scores > 0].std().item())

Images : torch.Size([4, 3, 512, 512])
Scores : torch.Size([4, 1, 512, 512])
Geos   : torch.Size([4, 8, 512, 512])
Nmaps  : torch.Size([4, 1, 512, 512])
Image range: -2.1179039478302 to 2.640000104904175
Score unique values: tensor([0., 1.])
Geo stats: mean 0.0132293701171875 std 7.47803258895874
Nmap stats (on positive pixels): mean 127.58235168457031 std 35.18562698364258


In [ ]:
import torch
from src.model import EAST

model = EAST(pretrained=False)
dummy = torch.randn(1, 3, 512, 512)  # batch of one
score, geo = model(dummy)

print("Score map:", score.shape)  # expect [1, 1, 512, 512]
print("Geo map:", geo.shape)      # expect [1, 8, 512, 512]

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\faizan/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100.0%


Score map: torch.Size([1, 1, 512, 512])
Geo map: torch.Size([1, 8, 512, 512])


In [ ]:
import torch
from src.model import EAST
from src.losses import EASTLoss

# Assuming imgs, scores, geos, and nmaps are already loaded from your DataLoader test.
# If not, you'd need to run the DataLoader test cell again to get these variables.
# You might want to move these to a CUDA device if available.
# imgs = imgs.to('cuda')
# ... and so on for the other tensors

# Instantiate the model and criterion
model = EAST(pretrained=True)  # Use pretrained=True to follow the paper's method
model.eval()  # Set model to evaluation mode for a clean forward pass test

# Perform a forward pass to get predictions
with torch.no_grad():
    pred_score, pred_geo = model(imgs)

# Instantiate the loss function
criterion = EASTLoss()

# Compute the loss using your predictions and ground truth maps
total_loss, score_loss, geo_loss = criterion(pred_score, pred_geo, scores, geos, nmaps)

# Print the results
print("Total loss:", total_loss.item())
print("Score loss:", score_loss.item())
print("Geo loss  :", geo_loss.item())

Total loss: 0.1503923237323761
Score loss: 0.020851217210292816
Geo loss  : 0.12954111397266388


In [1]:
!git clone https://github.com/SKfaizan-786/EAST-text-detection.git
%cd EAST-text-detection

Cloning into 'EAST-text-detection'...
remote: Enumerating objects: 7704, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 7704 (delta 15), reused 16 (delta 5), pack-reused 7670 (from 1)
Receiving objects: 100% (7704/7704), 307.15 MiB | 36.85 MiB/s, done.
Resolving deltas: 100% (4647/4647), done.
Updating files: 100% (7515/7515), done.
/content/EAST-text-detection


In [ ]:
!python -m src.train

Train size: 900,  Val size: 100
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100% 44.7M/44.7M [00:00<00:00, 118MB/s]
Training on 900 images and validating on 100 images for 100 epochs
Epoch 1/100: 100%|█| 225/225 [02:42<00:00,  1.39it/s, total=0.1345, score=0.0551
Epoch 1 | train loss 0.0950 | val loss 0.1001
✅ New best model saved with val loss: 0.1001
Epoch 2/100: 100%|█| 225/225 [02:38<00:00,  1.42it/s, total=0.1810, score=0.0808
Epoch 2 | train loss 0.0945 | val loss 0.0834
✅ New best model saved with val loss: 0.0834
Epoch 3/100: 100%|█| 225/225 [02:31<00:00,  1.49it/s, total=0.0751, score=0.0176
Epoch 3 | train loss 1476.6351 | val loss 0.0967
Epoch 4/100: 100%|█| 225/225 [02:24<00:00,  1.56it/s, total=0.1641, score=0.0456
Epoch 4 | train loss 0.0978 | val loss 0.0920
Epoch 5/100: 100%|█| 225/225 [02:20<00:00,  1.60it/s, total=0.4534, score=0.0714
Epoch 5 | train loss 2411.1412 | val loss 0.39

In [2]:
!python -m src.train

Train size: 900,  Val size: 100
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100% 44.7M/44.7M [00:00<00:00, 143MB/s]
Training on 900 images | validating on 100 images
Device: cuda
Epoch 1/100: 100%|█| 225/225 [03:02<00:00,  1.23it/s, total=0.1162, score=0.0350
Epoch 1 | train 0.0956 | val 0.0879
✅ New best model saved with val loss 0.0879
Epoch 2/100: 100%|█| 225/225 [02:56<00:00,  1.27it/s, total=0.0581, score=0.0078
Epoch 2 | train 0.0919 | val 0.0900
Epoch 3/100: 100%|█| 225/225 [02:49<00:00,  1.33it/s, total=0.0782, score=0.0142
Epoch 3 | train 0.0868 | val 0.0858
✅ New best model saved with val loss 0.0858
Epoch 4/100: 100%|█| 225/225 [02:41<00:00,  1.39it/s, total=0.2410, score=0.0233
Epoch 4 | train 0.0843 | val 0.0718
✅ New best model saved with val loss 0.0718
Epoch 5/100: 100%|█| 225/225 [02:37<00:00,  1.43it/s, total=0.0359, score=0.0037
Epoch 5 | train 0.0866 | val 0.0801
Epoch 6/100: 10